In [1]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

import os
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import json


# from google.colab import drive
# drive.mount('/content/drive')
# just use df = pd.read_csv('st1_data_train_sent.csv') when running in the repo
df = pd.read_csv('/content/drive/My Drive/UP_w22/PM/task 8/data/st1_data_train_sent.csv')


In [7]:
df = df.drop(['Unnamed: 0', 'post_id', 'subreddit_id', 'Components'], axis=1)
df

,Sentence,Label
0,De-Nial\nI wrote this a few years ago and just...,none
1,I thought I'd share...\n\n&#x200B;\n\nWhen I w...,none
2,Like the opening line of the old Frank Sanatra...,none
3,"I was getting ready to graduate high school, I...",none
4,Growing up I was an only child with older pare...,none
...,...,...
127354,Just how effective these drugs are and how do ...,question
127355,Edit2: Thank you.,none
127356,It is wonderful that there are drugs that can ...,none
127357,I am glad that Trikafta exists NOW.,none


In [8]:
df = df.rename(columns={"Sentence": "text", "Label": "label"})
df

,text,label
0,De-Nial\nI wrote this a few years ago and just...,none
1,I thought I'd share...\n\n&#x200B;\n\nWhen I w...,none
2,Like the opening line of the old Frank Sanatra...,none
3,"I was getting ready to graduate high school, I...",none
4,Growing up I was an only child with older pare...,none
...,...,...
127354,Just how effective these drugs are and how do ...,question
127355,Edit2: Thank you.,none
127356,It is wonderful that there are drugs that can ...,none
127357,I am glad that Trikafta exists NOW.,none


In [9]:
labels = ['question', 'claim', 'claim_per_exp', 'per_exp']

for idx, row in df.iterrows(): 
  text = df.loc[idx, 'text']
  label = df.loc[idx, 'label']
  if label in labels:
    df.loc[idx, 'label'] = labels.index(label) +1
  else:
    df.loc[idx, 'label'] = 0
    
df

,text,label
0,De-Nial\nI wrote this a few years ago and just...,0
1,I thought I'd share...\n\n&#x200B;\n\nWhen I w...,0
2,Like the opening line of the old Frank Sanatra...,0
3,"I was getting ready to graduate high school, I...",0
4,Growing up I was an only child with older pare...,0
...,...,...
127354,Just how effective these drugs are and how do ...,1
127355,Edit2: Thank you.,0
127356,It is wonderful that there are drugs that can ...,0
127357,I am glad that Trikafta exists NOW.,0


In [10]:
df['label'].describe()

count     127359
unique         5
top            0
freq      107681
Name: label, dtype: int64

In [11]:
# remove this cell if imbalanced data is wanted
smallest = 100000
i_len = []

for i in [0,1,2,3,4]:
  new_len = len(df[df["label"]==i])
  i_len.append(new_len)
  if new_len < smallest:
    smallest = new_len

df_four = df[df["label"]==4]
df_four = df_four[:smallest]
df_three = df[df["label"]==3]
df_three = df_three[:smallest]
df_two = df[df["label"]==2]
df_two = df_two[:smallest]
df_one = df[df["label"]==1]
df_one = df_one[:smallest]
df_zero = df[df["label"]==0]
df_zero = df_zero[:smallest]

df_balanced = pd.concat([df_zero, df_one, df_two, df_three, df_four])
df_balanced = df_balanced.reset_index(drop=True)

df = df_balanced.sample(frac=1).reset_index(drop=True)
df
print(i_len)

[107681, 6503, 579, 2389, 10207]


In [ ]:
df['label'].describe()

count     2895
unique       5
top          0
freq       579
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)
print(len(df_train),len(df_test)) 


2316 579


In [ ]:
print(len(df_train[df_train['label']==1])/len(df_train[df_train['label']==0]))
print(len(df_train[df_train['label']==2])/len(df_train[df_train['label']==0]))
print(len(df_train[df_train['label']==3])/len(df_train[df_train['label']==0]))
print(len(df_train[df_train['label']==4])/len(df_train[df_train['label']==0]))

0.9501039501039501
0.9646569646569647
0.9708939708939709
0.9293139293139293


In [ ]:
print(len(df_test[df_test['label']==1])/len(df_test[df_test['label']==0]))
print(len(df_test[df_test['label']==2])/len(df_test[df_test['label']==0]))
print(len(df_test[df_test['label']==3])/len(df_test[df_test['label']==0]))
print(len(df_test[df_test['label']==4])/len(df_test[df_test['label']==0]))

1.2448979591836735
1.1734693877551021
1.1428571428571428
1.346938775510204


In [ ]:
df_test

,text,label
968,I have so much stuff to do before my move in t...,4
1103,What should I be talking with my new doctor ab...,1
812,"Just out of pure curiosity, whats the highest ...",1
1632,I was told that the first ten years were the w...,0
1128,26M currently dealing with my first ever Gout ...,0
...,...,...
384,While the Rituxan has held the line in the sen...,0
2058,I'm not a fan of taking drugs as the side effe...,0
1569,First year I stupidly tried to mitigate with d...,3
2113,So I researched it online and apparently the o...,2


In [ ]:
df = DatasetDict()

df['train'] = Dataset.from_pandas(df_train)
df['test'] = Dataset.from_pandas(df_test)

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 2316
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 579
    })
})

In [ ]:
df['train'][151]

{'text': 'Doing great - update\nWanted to give a little update as it has been about 5 months since being diagnosed with gout.',
 'label': 4,
 '__index_level_0__': 2263}

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_df = df.map(preprocess_function, batched=True)

Map:   0%|          | 0/2316 [00:00<?, ? examples/s]

Map:   0%|          | 0/579 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "none", 1: "question", 2: "claim", 3: "claim per experience", 4: "per experience"}
label2id = {"none": 0, "question": 1, "claim": 2, "claim per experience": 3, "per experience": 4}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [ ]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df["train"],
    eval_dataset=tokenized_df["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/Chung-Fan/my_model into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.057231,0.569948
2,No log,0.980725,0.623489


TrainOutput(global_step=290, training_loss=1.1237328891096443, metrics={'train_runtime': 49.6752, 'train_samples_per_second': 93.246, 'train_steps_per_second': 5.838, 'total_flos': 91864846133040.0, 'train_loss': 1.1237328891096443, 'epoch': 2.0})